In [1]:
import pandas as pd
import numpy as np
import json

# 身長,体重,ラベルのCSVデータを読み出す
csv = pd.read_csv("bmi.csv")
# ラベルを三次元のクラスで表す
bclass = {"thin": [1,0,0], "normal": [0,1,0], "fat": [0,0,1]}
csv["label_pat"] = csv["label"].apply(lambda x : np.array(bclass[x]))
# データを正規化
csv["height"] = csv["height"] / 200
csv["weight"] = csv["weight"] / 100
print(csv)


height  weight   label  label_pat
0       0.710    0.62     fat  [0, 0, 1]
1       0.710    0.73     fat  [0, 0, 1]
2       0.885    0.61  normal  [0, 1, 0]
3       0.935    0.48    thin  [1, 0, 0]
4       0.765    0.60     fat  [0, 0, 1]
...       ...     ...     ...        ...
19995   0.610    0.58     fat  [0, 0, 1]
19996   0.965    0.69  normal  [0, 1, 0]
19997   0.965    0.37    thin  [1, 0, 0]
19998   0.975    0.51    thin  [1, 0, 0]
19999   0.815    0.67  normal  [0, 1, 0]

[20000 rows x 4 columns]


In [2]:
step=0
rows = csv[step*10:(step+1)*10]
x_pat = rows[["weight","height"]]
y_ans = rows["label_pat"]

csv[0:0]

,height,weight,label,label_pat


In [3]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [9]:
step = 0
rows = csv[1+step*10:(step+1)*10]
x_pat = rows[["weight","height"]]
y_ans = list(rows["label_pat"])

x = tf.placeholder(tf.float32, [None, 2], name="x")
y_ = tf.placeholder(tf.float32, [None, 3], name="y_")

sess = tf.Session()
#sess.run(x, feed_dict={x: x_pat})
sess.run(y_, feed_dict={y_: y_ans})




array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [4]:
# データを入れるプレースホルダを宣言
x    = tf.placeholder(tf.float32, [None, 2], name="x") # 身長,体重のデータを入れる
y_ = tf.placeholder(tf.float32, [None, 3]) # 答えのラベルを入れる

# 変数を宣言
W = tf.Variable(tf.random_uniform([2, 3], -1.0, 1.0)); # 重み
b = tf.Variable(tf.zeros([3])); # バイアス
# ソフトマックス回帰を定義
y = tf.nn.softmax(tf.matmul(x, W) + b)

# モデルを訓練する
cross_entropy = -tf.reduce_sum(y_ * tf.log(tf.clip_by_value(y, 1e-10,1.0)))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cross_entropy)

# 正解率を求めるためにテストデータを準備
test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])
# 正解率を求める
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
acc = tf.reduce_mean(tf.cast(predict, "float"))

# セッションを開始
sess = tf.Session()
sess.run(tf.initialize_all_variables()) #変数を初期化
# テストデータを用いて学習させる
for step in range(5000):
    i = (step * 100) % 1000
    rows = csv[1 + i : 1 + i + 100]
    x_pat = rows[["weight","height"]]
    y_ans = list(rows["label_pat"])
    fd = {x: x_pat, y_: y_ans}
    sess.run(train, feed_dict=fd)
    if step % 100 == 0:
        e = sess.run(cross_entropy, feed_dict=fd)
        a = sess.run(acc, feed_dict={x: test_pat, y_: test_ans})
        print("step=", step, "ce=", e, "acc=", a)
        

Instructions for updating:
Use `tf.global_variables_initializer` instead.
step= 0 ce= 104.22368 acc= 0.4772
step= 100 ce= 79.69935 acc= 0.713
step= 200 ce= 68.7156 acc= 0.7812
step= 300 ce= 62.16098 acc= 0.8236
step= 400 ce= 57.63057 acc= 0.8516
step= 500 ce= 54.223156 acc= 0.871
step= 600 ce= 51.5192 acc= 0.8892
step= 700 ce= 49.293255 acc= 0.9008
step= 800 ce= 47.41143 acc= 0.9096
step= 900 ce= 45.78824 acc= 0.919
step= 1000 ce= 44.36596 acc= 0.9276
step= 1100 ce= 43.103897 acc= 0.9344
step= 1200 ce= 41.972347 acc= 0.9396
step= 1300 ce= 40.948956 acc= 0.9426
step= 1400 ce= 40.01657 acc= 0.9472
step= 1500 ce= 39.161705 acc= 0.9518
step= 1600 ce= 38.37357 acc= 0.9532
step= 1700 ce= 37.643414 acc= 0.9542
step= 1800 ce= 36.964085 acc= 0.957
step= 1900 ce= 36.329597 acc= 0.9586
step= 2000 ce= 35.734962 acc= 0.9608
step= 2100 ce= 35.175938 acc= 0.9624
step= 2200 ce= 34.64892 acc= 0.963
step= 2300 ce= 34.150787 acc= 0.9644
step= 2400 ce= 33.678864 acc= 0.9648
step= 2500 ce= 33.230766 acc= 0

In [186]:
# データを予測
test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
acc = tf.reduce_mean(tf.cast(predict, "float"))
r = sess.run(acc, feed_dict={x: test_pat, y_:test_ans})
print(r)

0.811


In [7]:
test = tf.argmax(y, 1)
r = sess.run(test, feed_dict={x: [[70/100, 170/200]]})
print(r)
                                                          

[1]
